In [ ]:
import os 
ct_fileroot = 'clinicaltrial_2021'
pharma_fileroot = 'pharma'


os.environ['ct_fileroot'] = ct_fileroot
os.environ['pharma_fileroot'] = pharma_fileroot

In [ ]:


ct_zip_path = "/tmp/clinicaltrial_2021.zip" #define clinicaltrial_2021 filepath
pharma_zip_path = "/tmp/pharma.zip" #define pharma fulepath

if os.path.exists(ct_zip_path):
    os.remove(ct_zip_path) #check if exists, delete
else:
    print("File not found:", ct_zip_path) #if not, print error message

if os.path.exists(pharma_zip_path): #check if exists
    os.remove(pharma_zip_path)
else:
    print("File not found:", pharma_zip_path) #if not, print error message

In [ ]:
dbutils.fs.ls('/FileStore/tables/') #list the files in the tables directory

[FileInfo(path='dbfs:/FileStore/tables/FaultDataset-1.csv', name='FaultDataset-1.csv', size=1703184, modificationTime=1680612896000),
 FileInfo(path='dbfs:/FileStore/tables/FaultDataset-2.csv', name='FaultDataset-2.csv', size=1703184, modificationTime=1682436722000),
 FileInfo(path='dbfs:/FileStore/tables/FaultDataset.csv', name='FaultDataset.csv', size=1703184, modificationTime=1678869219000),
 FileInfo(path='dbfs:/FileStore/tables/Occupancy_Detection_Data.csv', name='Occupancy_Detection_Data.csv', size=50968, modificationTime=1677674812000),
 FileInfo(path='dbfs:/FileStore/tables/RDD_practice-1.txt', name='RDD_practice-1.txt', size=105, modificationTime=1680525119000),
 FileInfo(path='dbfs:/FileStore/tables/RDD_practice.txt', name='RDD_practice.txt', size=105, modificationTime=1680524981000),
 FileInfo(path='dbfs:/FileStore/tables/TS021_2021_2.csv', name='TS021_2021_2.csv', size=497239, modificationTime=1682284783000),
 FileInfo(path='dbfs:/FileStore/tables/account-models/', name='ac

In [ ]:
dbutils.fs.cp('FileStore/tables/'+ ct_fileroot +'.zip', 'file:/tmp/') #copy the clinicaltrial_2021 zip file to the local file system

dbutils.fs.cp('/FileStore/tables/'+ pharma_fileroot+'.zip', 'file:/tmp/')#copy the pharma zip file to the local file system

True

In [ ]:
%sh
ls /tmp/  #use a shell command to list the files in the temporary directory 

Rserv
RtmplYXLe7
chauffeur-daemon-params
chauffeur-daemon.pid
chauffeur-env.sh
clinicaltrial_2021.zip
custom-spark.conf
driver-daemon-params
driver-daemon.pid
driver-env.sh
hsperfdata_root
pharma.zip
spark-2280570f-60a3-47bc-b8ec-f825c6382f4f
spark-71227765-1557-4b00-8c02-c4f668bd8d44
spark-87e26802-d79a-40de-a5bd-7fe34257fce3
spark-b96b379d-df45-423e-b24f-a2874fafacb5
systemd-private-05d6d521b1a24967869a290b0e7508ac-systemd-logind.service-6Eghvz
systemd-private-05d6d521b1a24967869a290b0e7508ac-systemd-resolved.service-hElWTy
tmp.dHMKOSJAS1
tmp9i2rjpst
tmpbx6tnitg
tmpw333vjm1
tmpyje6y646


In [ ]:
%sh
unzip -d /tmp/ /tmp/clinicaltrial_2021.zip #unzip the clinicaltrial_2021 file



Archive:  /tmp/clinicaltrial_2021.zip
  inflating: /tmp/clinicaltrial_2021.csv  


In [ ]:
%sh
unzip -d /tmp/ /tmp/pharma.zip #unzip the pharma file

Archive:  /tmp/pharma.zip
  inflating: /tmp/pharma.csv         


In [ ]:
dbutils.fs.mv('file:/tmp/clinicaltrial_2021.csv', '/FileStore/tables', True) #move the clinicaltrial file to the Filestore/tables directory
dbutils.fs.mv('file:/tmp/pharma.csv', '/FileStore/tables', True) #move the pharma file to the Filestore/tables directory

True

In [ ]:
clinicaltrial_filepath = '/FileStore/tables/clinicaltrial_2021.csv' #assign the filepath to a variable 

In [ ]:
pharma_filepath = '/FileStore/tables/pharma.csv' #assign the filepath to a variable

In [ ]:
dbutils.fs.head(clinicaltrial_filepath, 1000) #show the first 1000 bytes of the clinicaltrial file

[Truncated to first 1000 bytes]


'Id|Sponsor|Status|Start|Completion|Type|Submission|Conditions|Interventions\r\nNCT02758028|The University of Hong Kong|Recruiting|Aug 2005|Nov 2021|Interventional|Apr 2016||\r\nNCT02751957|Duke University|Completed|Jul 2016|Jul 2020|Interventional|Apr 2016|Autistic Disorder,Autism Spectrum Disorder|\r\nNCT02758483|Universidade Federal do Rio de Janeiro|Completed|Mar 2017|Jan 2018|Interventional|Apr 2016|Diabetes Mellitus|\r\nNCT02759848|Istanbul Medeniyet University|Completed|Jan 2012|Dec 2014|Observational|May 2016|Tuberculosis,Lung Diseases,Pulmonary Disease|\r\nNCT02758860|University of Roma La Sapienza|Active, not recruiting|Jun 2016|Sep 2020|Observational [Patient Registry]|Apr 2016|Diverticular Diseases,Diverticulum,Diverticulosis|\r\nNCT02757209|Consorzio Futuro in Ricerca|Completed|Apr 2016|Jan 2018|Interventional|Apr 2016|Asthma|Fluticasone,Xhance,Budesonide,Formoterol Fumarate,Salmeterol Xinafoate\r\nNCT02752438|Ankara University|Unknown status|May 2016|Jul 2017|Observationa

In [ ]:
pharma = spark.read.options(delimiter = ',', header = True).csv(pharma_filepath).select('Parent_Company') #read in the parent_company column


pharma.show(5)

+-------------------+
|     Parent_Company|
+-------------------+
|Abbott Laboratories|
|             AbbVie|
|             AbbVie|
|Abbott Laboratories|
|  Johnson & Johnson|
+-------------------+
only showing top 5 rows



In [ ]:

#read in the two files splitting at the delimiters
clinicaltrial_2021 = sc.textFile(clinicaltrial_filepath,)\
                        .map(lambda line: line.split("|"))\
                        .map(lambda values: (values[0], values[1], values[2], values[3], values[4], values[5], values[6], values[7], values[8]))

pharma_rdd = sc.textFile(pharma_filepath).map(lambda line: line.split(',')[1])
column_header = clinicaltrial_2021.first() #assign the column headers to the variable column_header
clinicaltrial_2021 = clinicaltrial_2021.filter(lambda x: x!=column_header) #assign the rest of the dataset 




clinicaltrial_2021.take(5) #show the first 5 rdds

[('NCT02758028',
  'The University of Hong Kong',
  'Recruiting',
  'Aug 2005',
  'Nov 2021',
  'Interventional',
  'Apr 2016',
  '',
  ''),
 ('NCT02751957',
  'Duke University',
  'Completed',
  'Jul 2016',
  'Jul 2020',
  'Interventional',
  'Apr 2016',
  'Autistic Disorder,Autism Spectrum Disorder',
  ''),
 ('NCT02758483',
  'Universidade Federal do Rio de Janeiro',
  'Completed',
  'Mar 2017',
  'Jan 2018',
  'Interventional',
  'Apr 2016',
  'Diabetes Mellitus',
  ''),
 ('NCT02759848',
  'Istanbul Medeniyet University',
  'Completed',
  'Jan 2012',
  'Dec 2014',
  'Observational',
  'May 2016',
  'Tuberculosis,Lung Diseases,Pulmonary Disease',
  ''),
 ('NCT02758860',
  'University of Roma La Sapienza',
  'Active, not recruiting',
  'Jun 2016',
  'Sep 2020',
  'Observational [Patient Registry]',
  'Apr 2016',
  'Diverticular Diseases,Diverticulum,Diverticulosis',
  '')]

In [ ]:
column_header #show the members of the column_header variable

('Id',
 'Sponsor',
 'Status',
 'Start',
 'Completion',
 'Type',
 'Submission',
 'Conditions',
 'Interventions')

('Id',
 'Sponsor',
 'Status',
 'Start',
 'Completion',
 'Type',
 'Submission',
 'Conditions',
 'Interventions')

In [ ]:

pharma_rdd.take(5) #

['"Parent_Company"', '"Abbott Laboratories"', '"AbbVie"', '"AbbVie"', ' Inc."']

In [ ]:
#Question 1 - Number of the distinct studies in the dataset
count = clinicaltrial_2021.count() #count the number of rows
print(f'The number of distinct studies in this dataset are {count}')

The number of distinct studies in this dataset are 387261


In [ ]:
#Question2 - The number of studies for each type of study


#creates an rdd for each group of unique elements, counts with the len function, sorts in descending order
counts = clinicaltrial_2021.groupBy(lambda x: x[5])\
.mapValues(lambda x: len(set(x)))\
.sortBy(lambda x: x[1], ascending=False)           
print(counts.collect())        

[('Interventional', 301472), ('Observational', 77540), ('Observational [Patient Registry]', 8180), ('Expanded Access', 69)]


In [ ]:
#Question 3 - group the number of studies by the type of condition and return in descending order

# Extract the "Conditions" column at index 7
conditions_rdd = clinicaltrial_2021.map(lambda cols: cols[7])

#split on the comma, ignore null values and map each element to a tuple
unique_conditions_rdd = conditions_rdd \
    .map(lambda row: row.split(",")) \
    .flatMap(lambda x: x) \
    .filter(lambda x: x != "") \
    .map(lambda x: (x, 1))

grouped_rdd = unique_conditions_rdd.reduceByKey(lambda a, b: a + b)

# Sort the result by frequency in descending order and take the top 5
sorted_rdd = grouped_rdd.sortBy(lambda x: x[1], ascending=False)
top_5 = sorted_rdd.take(5)

# Display the result
for row in top_5:
    print(row[0], row[1])

Carcinoma 13389
Diabetes Mellitus 11080
Neoplasms 9371
Breast Neoplasms 8640
Syndrome 8032


In [ ]:
pharma_rdd.take(5)

['"Parent_Company"', '"Abbott Laboratories"', '"AbbVie"', '"AbbVie"', ' Inc."']

In [ ]:
#Question 4 - return the number of studies carried out by the top 10 sponsors
sponsor_rdd = clinicaltrial_2021.map(lambda x: x[1]) #take the second element of each row 

non_pharma_rdd = sponsor_rdd.subtract(pharma_rdd.map(lambda x: x.strip('"'))) #strip by the inverted commas and remove the values present in pharma_rdd

non_pharma_rdd = non_pharma_rdd.map(lambda x: (x, 1)) #create pair rdds with the interger 1

rdd_count = non_pharma_rdd.reduceByKey(lambda a, b: a + b) #reduce to unique keys and add the associated values

sorted_rdd = rdd_count.sortBy(lambda x: x[1], ascending = False) #sort by the values i.e. second element of the tuple

sorted_rdd.take(10)

[('National Cancer Institute (NCI)', 3218),
 ('M.D. Anderson Cancer Center', 2414),
 ('Assistance Publique - Hôpitaux de Paris', 2369),
 ('Mayo Clinic', 2300),
 ('Merck Sharp & Dohme Corp.', 2243),
 ('Assiut University', 2154),
 ('Novartis Pharmaceuticals', 2088),
 ('Massachusetts General Hospital', 1971),
 ('Cairo University', 1928),
 ('Hoffmann-La Roche', 1828)]

In [ ]:
#question 5 -- Show the number of studies completed each month in 2021
# Filter by completion date in 2020
completed_2021rdd = clinicaltrial_2021.filter(lambda x: x[4][-4:] == '2021' and x[2] == 'Completed')

# Map the completion date to month and count each occurrence
month_count_rdd = completed_2021rdd.map(lambda x: (x[4][:3] + ' ' + x[4][-4:], 1)).reduceByKey(lambda x, y: x + y)

# Sort by frequency in descending order and take the top 10
sorted_rdd = month_count_rdd.sortBy(lambda x: x[1], ascending=False).take(10)

# Display the result
for row in sorted_rdd:
    print(row[0], "Completed", row[1])

    




Mar 2021 Completed 1227
Jan 2021 Completed 1131
Jun 2021 Completed 1094
May 2021 Completed 984
Apr 2021 Completed 967
Feb 2021 Completed 934
Jul 2021 Completed 819
Aug 2021 Completed 700
Sep 2021 Completed 528
Oct 2021 Completed 187


In [ ]:
#10 most common interventions in the clinical trials dataset
interventions_rdd = clinicaltrial_2021.map( lambda x: x[-1]).filter(lambda x: x != "").map(lambda row: row.split(",")).flatMap(lambda x: x) #take the value of each row
interventions_rdd2 = interventions_rdd.map(lambda x: (x, 1)) #pair in a tuple with value 1
interventions_count = interventions_rdd2.reduceByKey(lambda x, y: x+y) #reduce by adding the intergers in the tuplels
sorted_interventions = interventions_count.sortBy(lambda x: x[1], ascending = False) #sort in descending order
sorted_interventions.take(10) #display the top 10 


[('Paclitaxel', 3225),
 ('Cyclophosphamide', 3012),
 ('Dexamethasone', 2516),
 ('Carboplatin', 2392),
 ('Antibodies', 2335),
 ('Vaccines', 2214),
 ('Gemcitabine', 2195),
 ('Bevacizumab', 2005),
 ('Docetaxel', 1953),
 ('Rituximab', 1740)]